In [ ]:
!pip install langchain langchain-community faiss-cpu pypdf sentence-transformers transformers accelerate bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 13.8 MB/s eta 0:00:00


In [ ]:
from google.colab import files
uploaded = files.upload()

pdf_path = list(uploaded.keys())[0]
pdf_path


Saving Research_Paper_27 (1).pdf to Research_Paper_27 (1) (2).pdf


'Research_Paper_27 (1) (2).pdf'

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(pdf_path)
docs = loader.load()

len(docs)


4

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=100)
chunks = splitter.split_documents(docs)

len(chunks)


23

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


/tmp/ipython-input-1855618056.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or 

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from langchain_community.vectorstores import FAISS

vector_store = FAISS.from_documents(chunks, embeddings)


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "Qwen/Qwen2.5-1.5B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16
)


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [ ]:
def answer_query(query):
    # retrieve relevant text
    docs = vector_store.similarity_search(query, k=3)
    context = "\n\n".join([d.page_content for d in docs])

    # create prompt
    prompt = f"""You are a helpful assistant.
Use the context to answer the question.

Context:
{context}

Question: {query}

Answer:"""

    # generate
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    output = model.generate(
        **inputs,
        max_new_tokens=200,
        temperature=0.2
    )

    return tokenizer.decode(output[0], skip_special_tokens=True)


In [ ]:
print(answer_query("Give me a summary in 5 bullet points."))


You are a helpful assistant.
Use the context to answer the question.

Context:
of precision ensures that critical meeting content is captured
without significant information loss [6], [7].
B. Summarization Quality
The abstractive summarization models mBART and mT5
generated coherent and concise summaries from long tran-
scripts. Key decisions, discussion highlights, and task items
were clearly extracted and listed in the structured output.
Compared to extractive methods, abstractive models provided
summaries that were closer to human-written notes, improving
readability and comprehension. In user evaluations, over 85%
of participants reported that the summaries were sufficiently
clear to replace manual note-taking. Additionally, the system
demonstrated capability for multilingual summarization, which
is critical in diverse organizational contexts [8], [9], [12].

sentences directly from the transcript, whereas abstractive
models generate human-like summaries by rephrasing content.
Adva

In [ ]:
while True:
    q = input("Ask a question (or exit): ")
    if q.lower() == "exit":
        break
    print(answer_query(q))


Ask a question (or exit): Give me a 5-point summary of this report.
You are a helpful assistant.
Use the context to answer the question.

Context:
of precision ensures that critical meeting content is captured
without significant information loss [6], [7].
B. Summarization Quality
The abstractive summarization models mBART and mT5
generated coherent and concise summaries from long tran-
scripts. Key decisions, discussion highlights, and task items
were clearly extracted and listed in the structured output.
Compared to extractive methods, abstractive models provided
summaries that were closer to human-written notes, improving
readability and comprehension. In user evaluations, over 85%
of participants reported that the summaries were sufficiently
clear to replace manual note-taking. Additionally, the system
demonstrated capability for multilingual summarization, which
is critical in diverse organizational contexts [8], [9], [12].

and volunteers, provided qualitative feedback on the sys

In [ ]:
response = answer_query("Give me summary in 5 bullet points.")
print(response)


You are a helpful assistant.
Use the context to answer the question.

Context:
of precision ensures that critical meeting content is captured
without significant information loss [6], [7].
B. Summarization Quality
The abstractive summarization models mBART and mT5
generated coherent and concise summaries from long tran-
scripts. Key decisions, discussion highlights, and task items
were clearly extracted and listed in the structured output.
Compared to extractive methods, abstractive models provided
summaries that were closer to human-written notes, improving
readability and comprehension. In user evaluations, over 85%
of participants reported that the summaries were sufficiently
clear to replace manual note-taking. Additionally, the system
demonstrated capability for multilingual summarization, which
is critical in diverse organizational contexts [8], [9], [12].

sentences directly from the transcript, whereas abstractive
models generate human-like summaries by rephrasing content.
Adva